In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
import gzip
from subprocess import check_output
import math
%matplotlib inline

In [ ]:
production_df = pd.read_excel('../data/tmdb_production_data.xlsx', index_col=0)
movie_df = pd.read_excel('../data/tmdb_movie_data.xlsx', index_col=0)
genre_df = pd.read_excel('../data/movie_genres_data.xlsx', index_col=0)

In [ ]:
genre_df2 = genre_df

In [ ]:
genre_list = ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']

In [ ]:
revenue = production_df[['movie_title', 'movie_revenue', 'movie_budget', 'movie_profits']]
revenue = revenue[revenue['movie_revenue'] != 0]
revenue = revenue[revenue['movie_budget'] != 0]
revenue = revenue.drop(columns={'movie_revenue', 'movie_budget'})
revenue

In [ ]:
movie_ids = movie_df[['movie_id', 'movie_title']]
movie_ids

In [ ]:
revenue_ids = revenue.merge(movie_ids)
revenue_ids = revenue_ids[revenue_ids['movie_profits'] != 0]
revenue_ids = revenue_ids.drop(columns={'movie_title'})
revenue_ids

In [ ]:
genre_df = genre_df.drop(columns={'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'})
genre_df

In [ ]:
x=0
genre_count=[]
while x < len(genre_df):
    genre_count.append(len(genre_df['genre'][x].split(',')))
    x=x+1
genre_count

In [ ]:
genre_df['genre_count'] = genre_count
genre_df

In [ ]:
genre_count = revenue_ids.merge(genre_df)
genre_count = genre_count.drop(columns={'movie_id', 'genre'})
genre_count['movie_profits'] = round(genre_count['movie_profits']/1000000, 0)
genre_count

In [ ]:
box_data = genre_count

In [ ]:
movie_count = genre_count.groupby('genre_count').count()
movie_count = movie_count.rename(columns={'movie_profits':'movie_count'})
movie_count

In [ ]:
genre_count = genre_count.groupby('genre_count').mean()

In [ ]:
data = genre_count.join(movie_count)
data = data.reset_index()
data = data.rename(columns={'genre_count':'Number of Genres', 'movie_profits':'Average Profits (Millions)', 'movie_count':'Number of Movies'})
data = data[data['Number of Movies'] > 50]
data

In [ ]:
means = list(data['Average Profits (Millions)'])
means = [int(i) for i in means]
means

In [ ]:
plt.figure(figsize=(14,11))
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")

bar = sns.barplot(x='Number of Genres', y='Average Profits (Millions)', data=data, color='#F9C31A', ci=False);

for x in range(len(means)):
    bar.text(x,means[x]+2.5, str(round(means[x],0)),
            fontdict = dict(color='white', fontsize=18),
            horizontalalignment = 'center')

In [ ]:
box_data = box_data[box_data['genre_count']!=7]
box_data = box_data[box_data['genre_count']!=6]
box_data['movie_profits'] = box_data['movie_profits']/1000
box_data

In [ ]:
plt.figure(figsize=(25,11))
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")

box = sns.boxplot(y='genre_count', x='movie_profits', data=box_data, color='#F9C31A',  orient='h');
box.set(ylabel=('Number of Genres'), xlabel=('Movie Profits (Billions)'))

In [ ]:
multi_genre = genre_df.merge(genre_df2)
multi_genre = multi_genre.drop(columns={'movie_id', 'genre'})
multi_genre = multi_genre[multi_genre['genre_count']==5]
multi_genre

In [ ]:
genres_5_count=[]

for x in genre_list:
    genres_5_count.append(multi_genre[x].value_counts()[1])
genres_5_count

In [ ]:
total_movies = 139

In [ ]:
data = {'Genres':genre_list, 'Number of Movies':genres_5_count}
genres_5_df = pd.DataFrame(data)
genres_5_df = genres_5_df.sort_values(by=['Number of Movies'], ascending=False)
genres_5_df['Percentage'] = round(genres_5_df['Number of Movies']/695, 2)
genres_5_df

In [ ]:
Percentage = genres_5_df['Percentage'].head(5).values.tolist()
Genre = genres_5_df['Genres'].head(5).values.tolist()

In [ ]:
Percentage.append(genres_5_df['Percentage'].tail(13).sum())
Genre.append('Other')

In [ ]:
Percentage

In [ ]:
plt.figure(figsize=(14,11))
sns.set(style="ticks", context="talk")
plt.style.use("dark_background")

colors = ['#a82200','#2a6671','#f7b23b','#3e6200', '#c05236', '#391513']

plt.pie(data=genres_5_df, x=Percentage, labels=Genre, startangle=65, colors=colors, autopct='%.0f%%');